## Source
https://github.com/fidalgobr/bibliaAveMariaJSON/blob/main/bibliaAveMaria.json

In [1]:
import json
import pandas as pd

with open('bibliaAveMaria.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def extract_structure(obj):
    if isinstance(obj, dict):
        return {k: extract_structure(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        if obj:
            return [extract_structure(obj[0])]
        else:
            return []
    else:
        return type(obj).__name__

structure = extract_structure(data)
# print(json.dumps(structure, indent=2, ensure_ascii=False))

In [2]:
# Extract only the nome of books. nome is inside novoTestamento and antigoTestamento
def extract_book_names(data):
    book_names = []
    if 'novoTestamento' in data:
        for book in data['novoTestamento']:
            book_names.append(book['nome'])
    if 'antigoTestamento' in data:
        for book in data['antigoTestamento']:
            book_names.append(book['nome'])
    return book_names

book_names = extract_book_names(data)
# print(json.dumps(book_names, indent=2, ensure_ascii=False))

In [3]:
# id -> mapped name (from your provided list)
book_id_map = {
    1001: "Gênesis",
    1002: "Êxodo",
    1003: "Levítico",
    1004: "Números",
    1005: "Deuteronômio",
    1006: "Josué",
    1007: "Juízes",
    1008: "Rute",
    1009: "I Samuel",
    1010: "II Samuel",
    1011: "I Reis",
    1012: "II Reis",
    1013: "I Crônicas",
    1014: "II Crônicas",
    1015: "Esdras",
    1016: "Neemias",
    1017: "Tobias",
    1018: "Judite",
    1019: "Ester",
    1020: "Ester",
    1021: "Jó",
    1022: "Salmos",
    1024: "I Macabeus",
    1025: "II Macabeus",
    1028: "Provérbios",
    1029: "Eclesiastes",
    1030: "Cântico dos Cânticos",
    1031: "Sabedoria",
    1032: "Eclesiástico",
    1033: "Isaías",
    1034: "Jeremias",
    1035: "Lamentações",
    1036: "Baruc",
    1038: "Ezequiel",
    1040: "Daniel",
    1041: "Oséias",
    1042: "Joel",
    1043: "Amós",
    1044: "Abdias",
    1045: "Jonas",
    1046: "Miquéias",
    1047: "Naum",
    1048: "Habacuc",
    1049: "Sofonias",
    1050: "Ageu",
    1051: "Zacarias",
    1052: "Malaquias",
    1053: "São Mateus",
    1054: "São Marcos",
    1055: "São Lucas",
    1056: "São João",
    1057: "Atos dos Apóstolos",
    1058: "Romanos",
    1059: "I Coríntios",
    1060: "II Coríntios",
    1061: "Gálatas",
    1062: "Efésios",
    1063: "Filipenses",
    1064: "Colossenses",
    1065: "I Tessalonicenses",
    1066: "II Tessalonicenses",
    1067: "I Timóteo",
    1068: "II Timóteo",
    1069: "Tito",
    1070: "Filêmon",
    1071: "Hebreus",
    1072: "São Tiago",
    1073: "I São Pedro",
    1074: "II São Pedro",
    1075: "I São João",
    1076: "II São João",
    1077: "III São João",
    1078: "São Judas",
    1079: "Apocalipse"
}

book_name_to_id = {v: k for k, v in book_id_map.items()}

In [ ]:
def flatten_verses(data):
    rows = []
    for testament in ['antigoTestamento', 'novoTestamento']:
        testament_name = 'old' if testament == 'antigoTestamento' else 'new'

        for book in data.get(testament, []):
            book_name = book['nome']
            for chapter in book['capitulos']:
                chapter_num = chapter['capitulo']
                for verse in chapter['versiculos']:
                    verse_num = verse['versiculo']
                    verse_text = verse['texto']
                    rows.append({
                        'testament': testament_name,
                        'book': book_name,
                        'book_id': book_name_to_id.get(book_name, None),
                        'chapter': chapter_num,
                        'number': verse_num,
                        'text': verse_text
                    })
    return pd.DataFrame(rows)

# Usage
with open('bibliaAveMaria.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = flatten_verses(data)
df.head()

,testament,book,book_id,chapter,verse,text
0,old,Gênesis,1001,1,1,"No princípio, Deus criou o céu e a terra."
1,old,Gênesis,1001,1,2,A terra estava sem forma e vazia; as trevas co...
2,old,Gênesis,1001,1,3,Deus disse: “Faça-se a luz!”. E a luz foi feita.
3,old,Gênesis,1001,1,4,"Deus viu que a luz era boa, e separou a luz da..."
4,old,Gênesis,1001,1,5,"Deus chamou à luz dia, e às trevas noite. Sobr..."


In [ ]:
# Save the DataFrame to a CSV file with this coluns: book_id, version_abbreviation, chapter, number, text

df['version_abbreviation'] = 'avm'
df.to_csv('avm.csv', index=False, sep=';', columns=['book_id', 'version_abbreviation', 'chapter', 'number', 'text'], encoding='utf-8')
